In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [ ]:
pathMTB = "drive/MyDrive/MTB/"
pathROAD = "drive/MyDrive/ROAD/"
pathCITY = "drive/MyDrive/CITY/"

In [ ]:
shapes = set()
for fileName in os.listdir(pathMTB):
  f = cv2.imread(pathMTB + fileName)
  if not f is None:
    shapes.add(f.shape)
for fileName in os.listdir(pathROAD):
  f = cv2.imread(pathROAD + fileName)
  if not f is None:
    shapes.add(f.shape)
for fileName in os.listdir(pathCITY):
  f = cv2.imread(pathCITY + fileName)
  if not f is None:
    shapes.add(f.shape)

In [ ]:
len(shapes)

In [ ]:
minX = 10000
minY = 10000
Zs = set()
for x,y,z in shapes:
  minX = min(x, minX)
  minY = min(y, minY)
  Zs.add(z)
minX, minY, Zs

In [ ]:
#final size of images
minX = 160
minY = 160

In [ ]:
MTB = []
ROAD = []
CITY = []
dim = (minX, minY)
for fileName in os.listdir(pathMTB):
  image = cv2.imread(pathMTB + fileName)
  if image is None:
    continue
  image = cv2.resize(src = image, dsize = dim, interpolation = cv2.INTER_LANCZOS4)
  MTB.append(np.array(image/255., dtype = np.float16))


for fileName in os.listdir(pathROAD):
  image = cv2.imread(pathROAD + fileName)
  if image is None:
    continue
  image = cv2.resize(src = image, dsize = dim, interpolation = cv2.INTER_LANCZOS4)
  ROAD.append(np.array(image/255., dtype = np.float16))

for fileName in os.listdir(pathCITY):
  image = cv2.imread(pathCITY + fileName)
  if image is None:
    continue
  image = cv2.resize(src = image, dsize = dim, interpolation = cv2.INTER_LANCZOS4)
  CITY.append(np.array(image/255., dtype = np.float16))

In [ ]:
MTB = np.array(MTB, dtype = np.float16)
ROAD = np.array(ROAD, dtype = np.float16)
CITY = np.array(CITY, dtype = np.float16)

In [ ]:
for i in range(5):
  plt.imshow(MTB[i].astype(np.float32))
  plt.show()
for i in range(5):
  plt.imshow(ROAD[i].astype(np.float32))
  plt.show()
for i in range(5):
  plt.imshow(CITY[i].astype(np.float32))
  plt.show()

In [ ]:
minAmount = min(MTB.shape[0], ROAD.shape[0], CITY.shape[0])
MTB.shape, ROAD.shape, CITY.shape, minAmount

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

In [ ]:
#Balanced dataset
X = tf.constant(np.concatenate((MTB[:minAmount], ROAD[:minAmount],CITY[:minAmount])), dtype = tf.float16)
Y = tf.constant(np.concatenate((np.zeros(minAmount), np.zeros(minAmount)+1, np.zeros(minAmount)+2)))
Y = tf.keras.utils.to_categorical(Y)

In [ ]:
#transfer learning example
resnet_model = keras.models.Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(160,160,3),
                   pooling='avg',classes=3,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)

resnet_model.add(keras.layers.Flatten())
resnet_model.add(keras.layers.Dense(512, activation='relu'))
resnet_model.add(keras.layers.Dense(3, activation='softmax'))
resnet_model.summary()

In [ ]:
#network from scratch
inputs = keras.Input(shape=(X.shape[1], X.shape[2], X.shape[-1]), dtype=tf.float16)

xl = layers.Conv2D(64, 3, activation="relu")(inputs)
xl = layers.Conv2D(64, 3, activation="relu")(xl)
xl = layers.Conv2D(64, 3, activation="relu")(xl)
xl = layers.MaxPooling2D(3)(xl)
xl = layers.Conv2D(32, 3, activation="relu")(xl)
xl = layers.Conv2D(32, 3, activation="relu")(xl)
xl = layers.Conv2D(32, 3, activation="relu")(xl)
xl = layers.MaxPooling2D(3)(xl)
xl = layers.Conv2D(16, 3, activation="relu")(xl)
xl = layers.Conv2D(16, 3, activation="relu")(xl)
xl = layers.Conv2D(16, 3, activation="relu")(xl)
xl = layers.MaxPooling2D(3)(xl)
#xl = layers.Conv2D(16, 3, activation="relu")(xl)
xl = layers.Flatten()(xl)
xl = dense = layers.Dense(64, activation="relu")(xl)
xl = dense = layers.Dense(8, activation="relu")(xl)
outputs = dense = layers.Dense(3)(xl)
resnet_model = keras.Model(inputs=inputs, outputs=outputs, name="bikes")
resnet_model.summary()

In [ ]:
resnet_model.compile(optimizer=keras.optimizers.Adam(0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
resnet_model.fit(X, Y, batch_size=64, epochs=5, validation_split = 0.1)